In [64]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    value type Data;
    value type 'Ground Station Ping';
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value out data:Data; 
            value in 'Ground Station Ping';
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    

    
    
    block 'Ground Station'{
        port CommIF : ~CommunicationInterface;      
        
        
        activity PingSpacecraft(){
            // 1. send signal on CommunicationLink interface 
           send 'Ground Station Ping'() to CommIF;
        } 
        
        activity ReceiveData(){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
            first start;
            then merge repeat;
            
            then accept trigger(dataFrame : Data);
            
            then action processData(argument flow from trigger::dataFrame);
            
            then repeat;
        }
        
        // Classifier Behaviour
        action ClassifierBehaviour_GS {
            first start;
            // wait 30secs
            then merge repeat;
                then action : PingSpacecraft();
                then action : ReceiveData();
            then repeat;
        }
    }
    
    
    
    
    block 'Spacecraft Vehicle'{
        port CommIF1 : CommunicationInterface;
       
        value type 'Battery Fully Charged';
        value type 'Shut Down Signal';
        value type 'Battery Low';
        value type 'Transmission Done Signal';
        
        
        activity ConsumePower(){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
            
            //this will run in the background (I hope)
            accept shutDown(shutDownSignal : 'Shut Down Signal');
            then done; //short for 'succession shutDown then done;'
            
            merge repeat;
            // battery--;
            then decide;
                if battery>= 40 then repeat;
                else sendBatteryLow;
                    
            send sendBatteryLow of 'Battery Low'() to Structure::Mission::spacecraftVehicle;
        }

        
        
        
        activity RechargeBatteries(){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1
            first start;
            then merge mergeNode;
            
            then decide;
                if battery>= 100 then sendCharged;
                else chargeBattery;
            
            action chargeBattery{ /* battery++; */ }
            then mergeNode;
            
            send sendCharged of 'Battery Fully Charged'() to CommIF1;
            then done;
        }
        
        activity SendDataFrames(){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
            
            first start;
            
            fork parallelStart;
            
            succession parallelStart then datasub;
            succession parallelStart then sendDataPacket;
            succession datasub then parallelEnd;
            succession sendDataPacket then parallelEnd;
            
            fork parallelEnd;
            then done;
            
            
            action datasub(){/* data = data-1024; */ }
            action sendDataPacket(){ send Data() to CommIF1; }
            
        }
        
        
        
        activity TransmitData(){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
            
            first start;
            then action js_1(){ /* javascript expression  */ }
            
            then fork fork1;
            succession fork1 then consumePower;
            succession fork1 then repeat;
                
            merge repeat;
            then decide;
                if data>0 then sendDataFrames;
                else js_2;
                
            action js_2(){ /* javascript expression  */ }
            then merge mergeNode_2;
            then send 'Shut Down Signal'() to Structure::Mission::spacecraftVehicle;
            then send 'Transmission Done Signal'() to Structure::Mission::spacecraftVehicle;
            
            
            action consumePower:ConsumePower;
            action sendDataFrames:SendDataFrames;
            then repeat;
        }
        
        action ClassifierBehaviour_SC {
            first start;
            action transmitting{
                
            }
            action recharging{
                
            }
        }
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

Package Structure (1a747a34)


In [50]:
%show Structure

Package Structure (39183b4f)
  [Import] Package ScalarValues (61f39bb)
  [Import] Package Interfaces (58bc5b30)
  [Membership] Package Interfaces (58bc5b30)
    [Membership] PortDefinition CommunicationInterface (6434da93)
      [FeatureMembership] ValueProperty data (480aafa6)
        [FeatureTyping] ValueType Data (76bd433)
        [Subsetting] ValueProperty values (44032fde)
      [Superclassing] PortDefinition Port (5a3a1bf9)
    [Membership] InterfaceDefinition CommunicationLink (2f7326e6)
      [EndFeatureMembership] PortUsage groundPort (318ad33f)
        [FeatureTyping] PortDefinition CommunicationInterface (6434da93)
        [Redefinition] PortUsage source (37c36608)
      [EndFeatureMembership] PortUsage spacePort (2f830326)
        [FeatureTyping] PortDefinition CommunicationInterface (6434da93)
        [Redefinition] PortUsage target (7e0bc8a3)
      [FeatureMembership] ItemFlow (6cd32961)
        [FeatureMembership] ItemFeature (2a8ea96d)
          [Redefinition] Feature i

In [63]:
%publish Structure

Saving element Package@773cf191 (Structure)
... element id is 4b5d6740-e0ac-11e9-9cc7-618afdf1f226
Saving element Package@61f39bb (ScalarValues)
... element id is 4b630c90-e0ac-11e9-9cc7-618afdf1f226
Saving element Package@4f82663e (Base)
... element id is 4b672b40-e0ac-11e9-9cc7-618afdf1f226
Saving element Comment@1ad8df52
... element id is 4b6ad4c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Annotation@a52ca2e
... relationship id is 4b6ea550-e0ac-11e9-9cc7-618afdf1f226
Saving element Comment@45d6ef73
... element id is 4b736040-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Annotation@3f29e26
... relationship id is 4b7730d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Membership@e04ccf8
... relationship id is 4b7ab340-e0ac-11e9-9cc7-618afdf1f226
Saving element Classifier@314c8b4a (Anything)
... element id is 4b7e83d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@1dd7796b
... relationship id is 4b825460-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Membership@

Saving element LiteralInteger@65a5d4f9
... element id is 4c8c4a50-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@4f37639
... relationship id is 4c8ff3d0-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@ceaca8 ($result)
... element id is 4c937640-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@f0985ec
... relationship id is 4c9746d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@3046fa0d
... relationship id is 4c9ac940-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@2dddc1b9
... relationship id is 4c9e4bb0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@10d4788a
... relationship id is 4ca1f530-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@7c2312fa
... relationship id is 4ca59eb0-e0ac-11e9-9cc7-618afdf1f226
Saving element Comment@5ff2e84b
... element id is 4ca92120-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Annotation@4cb0a000
... relationship id is 4caca390-e0ac-11e9-9cc7-618afdf1f22

... relationship id is 4da868b0-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@9e3df05 ($result)
... element id is 4dabc410-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@444f10aa
... relationship id is 4daf1f70-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@70b0220e
... relationship id is 4db2a1e0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@6ac0b715
... relationship id is 4db5fd40-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@4df39a88
... relationship id is 4dba1bf0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@7c75db8b
... relationship id is 4dbd9e60-e0ac-11e9-9cc7-618afdf1f226
Saving element Comment@3cd206b5
... element id is 4dc120d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Annotation@4ab550d5
... relationship id is 4dc47c30-e0ac-11e9-9cc7-618afdf1f226
Saving element Step@60acd609 (subperformances)
... element id is 4dc825b0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Feature

... relationship id is 4ec634c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@50448409
... relationship id is 4ec9b730-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Association@474fae39 (BinaryLink)
... relationship id is 4ece7220-e0ac-11e9-9cc7-618afdf1f226
Saving element Class@62ce72ff (Link)
... element id is 4ed21ba0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Superclassing@58a63629
... relationship id is 4ed59e10-e0ac-11e9-9cc7-618afdf1f226
Saving element Feature@2e1af49b (participant)
... element id is 4ed92080-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureTyping@3fd2322d
... relationship id is 4edcf110-e0ac-11e9-9cc7-618afdf1f226
Saving element LiteralUnbounded@1023150a
... element id is 4ee0c1a0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@509582eb
... relationship id is 4ee46b20-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@35dc3505 ($result)
... element id is 4ee862c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinit

... element id is 4fe25320-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@1f44fb73
... relationship id is 4fe5fca0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@509bc55d
... relationship id is 4fe97f10-e0ac-11e9-9cc7-618afdf1f226
Saving element LiteralInteger@7f608e21
... element id is 4fed2890-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@758f701b
... relationship id is 4ff0d210-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@24454a15 ($result)
... element id is 4ff45480-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@1ce2a28d
... relationship id is 4ff7d6f0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@7b30f2da
... relationship id is 4ffb5960-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@1a500561
... relationship id is 4ffeb4c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@64b012ef
... relationship id is 50023730-e0ac-11e9-9cc7-618afdf1f226
Saving relations

...

Saving relationship Redefinition@40a4dd1a
... relationship id is 7f29e6c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship EndFeatureMembership@de2f5a2
... relationship id is 7f2d9040-e0ac-11e9-9cc7-618afdf1f226
Saving element TargetEnd@21ac89fe
... element id is 7f3160d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@2c01b165
... relationship id is 7f34e340-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@52cf35bb
... relationship id is 7f38b3d0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship EndFeatureMembership@188dcb06
... relationship id is 7f3c3640-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@517f4e9a
... relationship id is 7f3fb8b0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@6d1c8d7c
... relationship id is 7f436230-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@17a2923a
... relationship id is 7f46e4a0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@c72cbfb
... relationship id is 7f4ab530-e0a

... relationship id is 805c9a60-e0ac-11e9-9cc7-618afdf1f226
Saving element ItemFeature@502f1683
... element id is 806043e0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@389005cc
... relationship id is 8063c650-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureMembership@2bb5e5d4
... relationship id is 80676fd0-e0ac-11e9-9cc7-618afdf1f226
Saving element InvocationExpression@76c0e848
... element id is 806bb590-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureTyping@725f2801
... relationship id is 806f5f10-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@1cf4da6b
... relationship id is 80730890-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@1bdf031e ($result)
... element id is 8076d920-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@1aede8cb
... relationship id is 807ad0c0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@210f2fa9
... relationship id is 807e5330-e0ac-11e9-9cc7-618afdf1f226
Saving relationship F

... element id is 8188be50-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureTyping@7bda5870
... relationship id is 818c40c0-e0ac-11e9-9cc7-618afdf1f226
Saving element LiteralInteger@286d6dd6
... element id is 818fea40-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@3aac0332
... relationship id is 8193bad0-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@266cd96d ($result)
... element id is 81976450-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@73ebd0cd
... relationship id is 819b0dd0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@795da961
... relationship id is 819ede60-e0ac-11e9-9cc7-618afdf1f226
Saving relationship FeatureValue@51005f09
... relationship id is 81a34b30-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@2b2fdfa2
... relationship id is 81a91790-e0ac-11e9-9cc7-618afdf1f226
Saving relationship BindingConnector@3d065ae1
... relationship id is 81b0dfc0-e0ac-11e9-9cc7-618afdf1f226
Saving element Feature@5

... relationship id is 82b97620-e0ac-11e9-9cc7-618afdf1f226
Saving element LiteralInteger@7a360554
... element id is 82bd1fa0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@9832715
... relationship id is 82c0c920-e0ac-11e9-9cc7-618afdf1f226
Saving element Parameter@6209f132 ($result)
... element id is 82c44b90-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Redefinition@54cc0e92
... relationship id is 82c7f510-e0ac-11e9-9cc7-618afdf1f226
Saving relationship ReturnParameterMembership@192aa863
... relationship id is 82cb7780-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Multiplicity@57562473
... relationship id is 82cef9f0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@4bc33720
... relationship id is 82d2a370-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@2dd0f797
... relationship id is 82d625e0-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsetting@4d18b73a
... relationship id is 82d9a850-e0ac-11e9-9cc7-618afdf1f226
Saving relationship Subsettin

Structure Thu Sep 26 22:23:41 UTC 2019 (4b594890-e0ac-11e9-9cc7-618afdf1f226)
